# Notebook de test 

#### Sommaire
<ol>
    <li ><a href=#Approche-non-supervisée> Approche non-supervisée</a></li>
    <ol>
        <li> <a href=#LatentDirichletAllocation>LatentDirichletAllocation</a></li>
        <li> <a href=#NonNegativeMatrixFactorisation>NonNegativeMatrixFactorisation</a></li>
    </ol>
    <li> <a href=#Approche-mixte> Approche mixte</a></li>
    <li> <a href=#Approche-supervisée> Approche supervisée</a></li>
    <li> <a href=#Modèle-retenu>Modèle retenu</a></li>
</ol>

## Import des bibliothèques

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from bs4 import BeautifulSoup
import xml
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD,LatentDirichletAllocation,NMF
from sklearn.cluster import DBSCAN,AgglomerativeClustering,SpectralClustering, AffinityPropagation
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.linear_model import PassiveAggressiveClassifier,Perceptron, SGDClassifier
from sklearn.naive_bayes import BaseNB,BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import resample
from joblib import Parallel, delayed
from itertools import product
from tensorflow.keras import layers,Model,Sequential,Input
from tensorflow.keras.layers import LSTM,Dense,MaxPool1D,RNN,Conv1D,Embedding,Conv2D,Flatten,SimpleRNNCell
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from numba import cuda
import time

In [23]:
query_df=pd.read_csv("exported_query_df.csv",index_col='Id')
corpus_by_tag_df=pd.read_csv("corpus_by_tag.csv")

In [24]:
corpus_by_tag_df.index=corpus_by_tag_df['Unnamed: 0']
corpus_by_tag_df.drop('Unnamed: 0',axis=1,inplace=True)

In [25]:
query_df.columns

Index(['CreationDate', 'AnswerCount', 'Body', 'CommentCount', 'FavoriteCount',
       'Score', 'Title', 'ViewCount', 'Body_text', 'Tag_list0', 'Tag_list1',
       'Tag_list2', 'Tag_list3', 'Tag_list4'],
      dtype='object')

In [26]:
query_df.drop(['AnswerCount','CommentCount', 'FavoriteCount',
       'Score','ViewCount',],axis=1,inplace=True)

In [27]:
time.ctime()[-4:]

'2021'

In [28]:
all_tag=[]
for i in query_df.columns:
    if 'Tag_l' in i :
        all_tag.extend(query_df[i].unique())
all_tag=set(all_tag)

In [29]:
len(all_tag-set(corpus_by_tag_df.index))
map_dict={i:np.nan for i in all_tag-set(corpus_by_tag_df.index)}

In [30]:
#for i in query_df.columns:
#    if 'Tag_l' in i :
#        query_df[i]=query_df[i].replace(map_dict)

In [31]:
#for k in np.arange(3):
#    print(k,'-iter')
#    try:
#        for i in query_df[[i for i in query_df.columns if 'Tag_l' in i]].columns:
#            print(i)
#            for j in query_df.index:
#                if query_df.loc[j,i] is np.nan:
#                    query_df.loc[j,i]=query_df.loc[j,i[:-1]+str(int(i[-1])+1)]
#    except:
#        pass


In [32]:
query_df[[i for i in query_df.columns if 'Tag_l' in i]].count()

Tag_list0    50856
Tag_list1    46844
Tag_list2    33805
Tag_list3    18125
Tag_list4     7012
dtype: int64

In [33]:
test_df=query_df[query_df.Tag_list0.notna()].copy()
test_df['Tag']=test_df['Tag_list0']
for i in query_df.columns:
    if 'Tag_l' in i and i[-1]!='0':
        x=query_df[query_df[i].notna()].copy()
        x['Tag']=x[i]
        test_df=pd.concat([test_df,x],axis=0)

In [34]:
84*1000-188*550

-19400

In [35]:
keys_to_keep=list(test_df.Tag.value_counts()[test_df.Tag.value_counts()>=500].keys())

In [36]:
test_df.Tag.value_counts()[test_df.Tag.value_counts()>=500].describe(percentiles=np.arange(50)/50)

count      45.000000
mean     1476.066667
std      1390.745735
min       503.000000
0%        503.000000
2%        509.160000
4%        519.880000
6%        525.560000
8%        531.160000
10%       536.600000
12%       545.440000
14%       567.440000
16%       596.960000
18%       618.080000
20%       629.600000
22%       633.360000
24%       634.000000
26%       654.240000
28%       687.360000
30%       705.200000
32%       715.760000
34%       735.120000
36%       737.680000
38%       753.840000
40%       779.800000
42%       794.920000
44%       810.320000
46%       836.640000
48%       850.280000
50%       889.000000
52%       945.320000
54%       953.000000
56%       953.640000
58.0%     959.720000
60%       973.400000
62%      1005.600000
64%      1063.360000
66%      1105.120000
68%      1173.760000
70%      1208.000000
72%      1385.000000
74%      1532.760000
76%      1714.480000
78%      1960.560000
80%      2160.400000
82%      2261.120000
84%      2295.440000
86%      2577

In [37]:
test_df[test_df.Tag=='javascript']

,CreationDate,Body,Title,Body_text,Tag_list0,Tag_list1,Tag_list2,Tag_list3,Tag_list4,Tag
Id,,,,,,,,,,
10826756,2012-05-31 02:35:09,<p>Here is a very simple program and the outpu...,JavaScript innerHTML not working,Here is a very simple program and the output s...,javascript,NaN,NaN,NaN,NaN,javascript
16998420,2013-06-08 10:13:08,<p>I am using dropzone.js for my drag-drop fil...,dropzone.js - how to do something after ALL fi...,I am using dropzone.js for my drag-drop file u...,javascript,jquery,dropzone.js,NaN,NaN,javascript
10830357,2012-05-31 08:57:24,<p>I am trying to convert Twitter datetime to ...,javascript toISOString() ignores timezone offset,I am trying to convert Twitter datetime to a l...,javascript,datetime,timezone-offset,NaN,NaN,javascript
9444745,2012-02-25 14:17:37,<p>I am trying to get JavaScript to display to...,JavaScript how to get tomorrows date in format...,I am trying to get JavaScript to display tomor...,javascript,NaN,NaN,NaN,NaN,javascript
9914216,2012-03-28 19:17:14,<p>This is my code so far:</p>\n\n<pre><code>v...,How do I separate an integer into separate dig...,This is my code so far:\nvar n = 123456789;\nv...,javascript,arrays,digits,NaN,NaN,javascript
...,...,...,...,...,...,...,...,...,...,...
26311470,2014-10-11 05:32:18,<p>I need to implement a function to run after...,What is the equivalent of javascript setTimeou...,I need to implement a function to run after 60...,java,javascript,timer,settimeout,setinterval,javascript
23101713,2014-04-16 06:30:14,"<p>I am working on a website, in which I have ...",How to open a link in new tab using javascript,"I am working on a website, in which I have to ...",c#,javascript,asp.net,NaN,NaN,javascript
21261850,2014-01-21 15:08:15,<p>I have a textbox on my website:</p>\n\n<pre...,Why does my save use the initial value of my T...,I have a textbox on my website:\n<asp:TextBox ...,c#,javascript,asp.net,NaN,NaN,javascript


In [38]:
test_df.index=np.arange(test_df.shape[0])

In [39]:
idx=[i for i in test_df.index if test_df.loc[i,'Tag'] in keys_to_keep]

In [40]:
test_df=test_df.loc[idx,:]

In [41]:
sampled_df=pd.DataFrame()
for i in test_df.Tag.unique():
    x=test_df[test_df.Tag==i].copy()
    x=resample(x,n_samples=2000)
    sampled_df=pd.concat([sampled_df,x],axis=0)

In [42]:
sampled_df.count()

CreationDate    90000
Body            90000
Title           90000
Body_text       90000
Tag_list0       90000
Tag_list1       85460
Tag_list2       66441
Tag_list3       39072
Tag_list4       16181
Tag             90000
dtype: int64

## Approche non-supervisée

In [43]:
sampled_df.shape

(90000, 10)

In [44]:
chr(114)

'r'

### LatentDirichletAllocation

In [45]:
lda=LatentDirichletAllocation(n_components=42)

In [46]:
custom_stop_word_list=list(TfidfVectorizer(stop_words='english').get_stop_words())
custom_stop_word_list.extend([chr(i) for i in np.arange(97,99)])
custom_stop_word_list.extend([chr(i) for i in np.arange(101,114)])
custom_stop_word_list.extend([chr(i) for i in np.arange(115,180)])
custom_stop_word_list.extend([str(i) for i in np.arange(0,180)])
custom_stop_word_list.extend([str(i) for i in [10,100,1000,10000,100000,1000000]])

In [47]:
tfidf=TfidfVectorizer(ngram_range=(1,2),stop_words=custom_stop_word_list,max_df=5/84.,min_df=1/(84*2),sublinear_tf=True)
tfidf_titre=TfidfVectorizer(max_df=1/52.,min_df=0.003,stop_words=custom_stop_word_list,sublinear_tf=True)

In [48]:
tf_df=tfidf.fit_transform(sampled_df.Body_text.apply(str))

In [49]:
tf_df.shape

(90000, 1003)

In [50]:
lda_df=lda.fit_transform(tf_df)

In [51]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 15
display_topics(lda, tfidf.get_feature_names(), no_top_words)


Topic 0:
div button div class input div div click li span input type href bootstrap btn form css label
Topic 1:
self bar foo view nil func controller let swift navigation button alloc yes storyboard override
Topic 2:
image color images background png img jpg blue red src background color white black css style
Topic 3:
console log console log item map parent items objects list child tags filter remove undefined string string
Topic 4:
xcode ios iphone device apple simulator build beta developer issue swift devices running launch storyboard
Topic 5:
visual visual studio studio vs google debug play microsoft 2012 services solution 2013 windows installed web
Topic 6:
std vector int program include main compiler linux namespace compile programming int main double memory char
Topic 7:
height size width view font scroll left position center screen layout right border margin background
Topic 8:
swift option objective nsstring select split typescript selected convert declare language classes let

### NonNegativeMatrixFactorisation

In [52]:
nmf =NMF(n_components=42,random_state=2)

In [53]:
nmf_df=nmf.fit_transform(tf_df)

In [54]:
display_topics(nmf, tfidf.get_feature_names(), no_top_words)


Topic 0:
page javascript jquery element document window event click browser load elements scroll body alert link
Topic 1:
js node node js npm module require express node_modules typescript npm install src index ts res modules
Topic 2:
table column insert rows row columns tables update delete query records primary users cell unique
Topic 3:
android studio android studio gradle build sdk tools google support xml visual studio visual device failed debug
Topic 4:
list item items elements element objects li contains names foreach ul values arrays convert simple
Topic 5:
df dataframe pandas column columns pd frame data frame index rows row values csv import names
Topic 6:
int char main include int main std printf void integer program pointer const vector length understand
Topic 7:
view self controller views storyboard yes nil alloc cell screen navigation action title func ios
Topic 8:
docker container running machine host build ubuntu port inside start ps connect ip sudo stop
Topic 9:
input 

## Approche mixte

## Approche supervisée

On va maintenant s'intéresser à des méthode supervisée pour classifier les textes. On va utiliser un nombre restreints de colonnes ainsi qu'une grille de recherche et des fonctions de préprocessing.

### Séparation des données et préparation des modèles.

In [55]:
query_df.columns


Index(['CreationDate', 'Body', 'Title', 'Body_text', 'Tag_list0', 'Tag_list1',
       'Tag_list2', 'Tag_list3', 'Tag_list4'],
      dtype='object')

In [56]:
sampled_df['year']=pd.to_datetime(sampled_df.CreationDate).dt.year

In [57]:
test_df['year']=pd.to_datetime(test_df.CreationDate).dt.year

In [58]:
non_word_column=['year']
words_columns=['Title','Body_text']
target_columns=['Tag_list0', 'Tag_list1',
       'Tag_list2', 'Tag_list3', 'Tag_list4']

In [59]:
lab_enc=LabelEncoder()

X=sampled_df[non_word_column+words_columns]
y=sampled_df['Tag']


In [60]:
y=lab_enc.fit_transform(y.fillna(''))

In [61]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [62]:
std=StandardScaler()
tsvd=TruncatedSVD(n_components=900, n_iter=20, random_state=2)
tsvd2=TruncatedSVD(n_components=180, n_iter=25, random_state=2)


word_proc=Pipeline([
    ('Tf-Idf',tfidf),
#    ('truncatedSVD_word',tsvd),
],verbose=True)
title_proc=Pipeline([
    ('Tf-Idf',tfidf_titre),
    ('truncatedSVD',tsvd2),
],verbose=True)


col_proc=ColumnTransformer([
#    ('Scale',std,non_word_column),
    ('Word_processing',word_proc,'Body_text'),
#    ('Title_processing',title_proc,'Title'),
],n_jobs=-3,verbose=True)

def pipe(model) :
    pipe=Pipeline([
#        ('Préparation des données',col_proc),
        (str(model)[:5],model),
    ],verbose=True)
    return pipe

In [63]:
a=tfidf.fit_transform(sampled_df.Body_text)

In [64]:
a.shape

(90000, 1003)

In [65]:
#a=tsvd.fit_transform(a)

In [66]:
#print(np.cumsum(np.round(tsvd.explained_variance_ratio_, decimals=4)*100))
#plt.plot(np.cumsum(np.round(tsvd.explained_variance_ratio_, decimals=4)*100))

In [67]:
def proba_label(model,X,n_class):
    x=pd.DataFrame(model.predict_proba(X))
    x.columns=lab_enc.inverse_transform(x.columns)
    z=pd.DataFrame(index=x.index,columns=[str(n_class)+' most likely tags'])
    z[str(n_class)+' most likely tags']=[
        {k:v for k,v in x.transpose()[i].sort_values(ascending=False)[:n_class].items()} for i in x.index
    ]
    return (x,z[str(n_class)+' most likely tags'])

In [68]:
#stratK=StratifiedKFold(n_splits=10,shuffle=True,random_state=2)
#kfolds=stratK.split(X_train,y_train)
#i=0
#train_folds=dict()
#test_folds=dict()
#train_folds_target=dict()
#test_folds_target=dict()
#for train,test in kfolds:
#    train_folds[i]=X_train.iloc[train,:]
#    test_folds[i]=X_train.iloc[test,:] 
#    train_folds_target[i]=y_train[train]
#    test_folds_target[i]=y_train[test] 
#    i+=1
#del i

In [69]:
#col_proc_folds=dict()
#prepared_train_folds=dict()
#for i in np.arange(len(train_folds)):
#    col_proc_folds[i]=ColumnTransformer([
#    ('Scale',std,non_word_column),
#    ('Word_processing',word_proc,'Body_text'),
#    ('Title_processing',title_proc,'Title'),
#],n_jobs=-3,verbose=True)
#    prepared_train_folds[i]=col_proc_folds[i].fit_transform(train_folds[i])
#    print('fold - %s prepared'%i)

In [70]:
#prepared_test_folds=dict()
#for i in np.arange(len(train_folds)):
#    prepared_test_folds[i]=col_proc_folds[i].transform(test_folds[i])
#    print('fold - %s prepared'%i)

In [71]:
"""def k_fold_gridsearch(estimator,kfoldtrain,kfoldtest,target_train,target_test,params,verbose,n_jobs):
    grid=dict()
    score=dict()
    param=dict()
    print('fitting %s folds for % candidates'
          %(len(kfoldtrain.keys()),np.prod([len(params[i].values()) for i in params.keys()]) )
    
    for key, values in params.items:
          
    for i in np.arange(len(params.keys())):
        grid[i]=GridSearchCV(estimator=estimator,param_grid=params,cv=5,n_jobs=n_jobs,verbose=verbose)
    print('grids instanciated')
    
          
    for i in np.arange(len(kfoldtrain)):
            grid[i].fit(kfoldtrain[i],target_train[i])
            print('grid - %s fitted'%i)
            param[i]=grid[i].best_params_
            score=grid[i].score(kfoldtest[i],target_test[i])
            print('score for %s is %s and parameters %s'%(i,score[i],param[i]))
    return grid,score,param"""

"def k_fold_gridsearch(estimator,kfoldtrain,kfoldtest,target_train,target_test,params,verbose,n_jobs):\n    grid=dict()\n    score=dict()\n    param=dict()\n    print('fitting %s folds for % candidates'\n          %(len(kfoldtrain.keys()),np.prod([len(params[i].values()) for i in params.keys()]) )\n    \n    for key, values in params.items:\n          \n    for i in np.arange(len(params.keys())):\n        grid[i]=GridSearchCV(estimator=estimator,param_grid=params,cv=5,n_jobs=n_jobs,verbose=verbose)\n    print('grids instanciated')\n    \n          \n    for i in np.arange(len(kfoldtrain)):\n            grid[i].fit(kfoldtrain[i],target_train[i])\n            print('grid - %s fitted'%i)\n            param[i]=grid[i].best_params_\n            score=grid[i].score(kfoldtest[i],target_test[i])\n            print('score for %s is %s and parameters %s'%(i,score[i],param[i]))\n    return grid,score,param"

In [72]:
startTime = time.time()
prepared_train_df=col_proc.fit_transform(X_train)
print('train prepared')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
startTime = time.time()
prepared_test_df=col_proc.transform(X_test)
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

train prepared
Execution time in seconds: 12.145272493362427
Execution time in seconds: 5.505429267883301


### Modèles.

#### Naive Base

In [73]:
nb=BernoulliNB()
pipebnb=pipe(nb,)

In [74]:
nb.fit(prepared_train_df,y_train)

BernoulliNB()

In [75]:
grid_bnb=GridSearchCV(nb,{'alpha': list(set(np.arange(1,3,0.01))),},cv=6,n_jobs=-3,verbose=5)


In [76]:
#pipebnb.fit(X_train.iloc[:],y_train[:])
grid_bnb.fit(prepared_train_df,y_train)

Fitting 6 folds for each of 200 candidates, totalling 1200 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-3)]: Done  60 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-3)]: Done 150 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-3)]: Done 276 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-3)]: Done 438 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-3)]: Done 636 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-3)]: Done 870 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-3)]: Done 1140 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-3)]: Done 1200 out of 1200 | elapsed:   46.2s finished


GridSearchCV(cv=6, estimator=BernoulliNB(), n_jobs=-3,
             param_grid={'alpha': [1.02, 1.0, 1.03, 1.01, 1.04, 1.05, 1.06,
                                   1.07, 1.08, 1.09, 1.3200000000000003,
                                   1.6400000000000006, 1.9600000000000009,
                                   2.030000000000001, 2.210000000000001,
                                   2.280000000000001, 2.3500000000000014,
                                   2.530000000000001, 2.6000000000000014,
                                   2.6700000000000017, 1.2500000000000002,
                                   1.5700000000000005, 1.8900000000000008,
                                   1.1800000000000002, 1.5000000000000004,
                                   1.8200000000000007, 2.990000000000002,
                                   2.140000000000001, 2.4600000000000013,
                                   2.7800000000000016, ...]},
             verbose=5)

In [77]:
grid_bnb.score(prepared_test_df,y_test)

0.5305555555555556

In [78]:
grid_bnb.best_params_

{'alpha': 1.03}

In [79]:
nb=BernoulliNB(alpha=0.61)


#### SGD

In [80]:
sgd=SGDClassifier(loss='modified_huber',n_jobs=-3,random_state=2,verbose=5)

In [81]:
sgd.fit(prepared_train_df,y_train)

[Parallel(n_jobs=-3)]: Using backend ThreadingBackend with 6 concurrent workers.


-- Epoch 1-- Epoch 1

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 12.21, NNZs: 1005, Bias: -1.605927, T: 72000, Avg. loss: 0.121402
Total training time: 0.03 seconds.
Norm: 14.25, NNZs: 1005, Bias: -2.054422, T: 72000, Avg. loss: 0.095997
Total training time: 0.02 seconds.
-- Epoch 2
-- Epoch 2
Norm: 14.58, NNZs: 1005, Bias: -1.794752, T: 72000, Avg. loss: 0.144513
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 12.30, NNZs: 983, Bias: -2.239116, T: 72000, Avg. loss: 0.052437
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 15.06, NNZs: 1002, Bias: -2.562640, T: 72000, Avg. loss: 0.077304
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 12.72, NNZs: 1003, Bias: -1.892873, T: 72000, Avg. loss: 0.145153
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 12.06, NNZs: 1005, Bias: -1.691151, T: 144000, Avg. loss: 0.050021
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 11.33, NNZs: 1005, Bias: -1.453907, T: 144000, Avg. loss: 0.072017
Total training time: 0.05 seconds.


[Parallel(n_jobs=-3)]: Done   6 tasks      | elapsed:    0.2s



Total training time: 0.15 seconds.

Total training time: 0.19 seconds.
Norm: 7.34, NNZs: 1005, Bias: -1.144019, T: 432000, Avg. loss: 0.071969-- Epoch 9
-- Epoch 7

Total training time: 0.17 seconds.
-- Epoch 7
Norm: 10.36, NNZs: 1005, Bias: -1.457212, T: 576000, Avg. loss: 0.047944
Total training time: 0.19 seconds.
-- Epoch 9
Norm: 8.70, NNZs: 1005, Bias: -1.116417, T: 504000, Avg. loss: 0.051967
Total training time: 0.16 seconds.
-- Epoch 8
Norm: 10.14, NNZs: 1005, Bias: -1.252561, T: 576000, Avg. loss: 0.046022
Total training time: 0.19 seconds.
-- Epoch 9
Norm: 9.23, NNZs: 1005, Bias: -1.361776, T: 648000, Avg. loss: 0.045775
Total training time: 0.22 seconds.
Convergence after 9 epochs took 0.22 seconds
Norm: 7.74, NNZs: 1005, Bias: -1.314459, T: 504000, Avg. loss: 0.056258
Total training time: 0.17 seconds.
-- Epoch 8
-- Epoch 1
Norm: 7.17, NNZs: 1005, Bias: -1.136184, T: 504000, Avg. loss: 0.071594
Total training time: 0.19 seconds.
-- Epoch 8
Norm: 10.29, NNZs: 1005, Bias: -1

-- Epoch 3
Norm: 8.47, NNZs: 1005, Bias: -1.399402, T: 288000, Avg. loss: 0.061395
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 5.24, NNZs: 1005, Bias: -1.026226, T: 432000, Avg. loss: 0.079161
Total training time: 0.14 seconds.
-- Epoch 7
Norm: 12.36, NNZs: 1005, Bias: -2.312562, T: 216000, Avg. loss: 0.039900
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 8.30, NNZs: 1005, Bias: -1.076797, T: 504000, Avg. loss: 0.061724
Total training time: 0.17 seconds.
-- Epoch 8
Norm: 7.31, NNZs: 1005, Bias: -1.088830, T: 504000, Avg. loss: 0.068065
Total training time: 0.16 seconds.
-- Epoch 8
Norm: 11.12, NNZs: 1004, Bias: -1.778064, T: 216000, Avg. loss: 0.038134
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 8.27, NNZs: 1005, Bias: -1.335957, T: 360000, Avg. loss: 0.060791
Total training time: 0.13 seconds.
-- Epoch 6
Norm: 11.47, NNZs: 1005, Bias: -2.204288, T: 288000, Avg. loss: 0.038745
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 7.23, NNZs: 1005, Bias: -1.080282, T

-- Epoch 3
Norm: 10.42, NNZs: 1005, Bias: -1.596894, T: 216000, Avg. loss: 0.029734
Total training time: 0.06 seconds.
-- Epoch 4
Norm: 11.12, NNZs: 1004, Bias: -1.679938, T: 216000, Avg. loss: 0.037867
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 7.55, NNZs: 1005, Bias: -1.118961, T: 432000, Avg. loss: 0.066402
Total training time: 0.12 seconds.
-- Epoch 7
Norm: 12.33, NNZs: 1005, Bias: -1.424788, T: 360000, Avg. loss: 0.040088
Total training time: 0.10 seconds.
-- Epoch 6
Norm: 6.86, NNZs: 1005, Bias: -1.103170, T: 504000, Avg. loss: 0.064253
Total training time: 0.13 seconds.
-- Epoch 8
Norm: 10.10, NNZs: 1005, Bias: -1.369020, T: 216000, Avg. loss: 0.052939
Total training time: 0.05 seconds.
-- Epoch 4
Norm: 10.02, NNZs: 1005, Bias: -1.518793, T: 288000, Avg. loss: 0.029272
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 10.61, NNZs: 1005, Bias: -1.602535, T: 288000, Avg. loss: 0.037151
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 7.42, NNZs: 1005, Bias: -1.102421

-- Epoch 6
Norm: 11.18, NNZs: 1005, Bias: -1.554833, T: 288000, Avg. loss: 0.046743
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 9.65, NNZs: 1005, Bias: -1.541691, T: 432000, Avg. loss: 0.039709
Total training time: 0.11 seconds.
-- Epoch 7
Norm: 9.15, NNZs: 1005, Bias: -1.484068, T: 288000, Avg. loss: 0.054746
Total training time: 0.07 seconds.
-- Epoch 5
Norm: 10.93, NNZs: 1005, Bias: -1.500063, T: 360000, Avg. loss: 0.046216
Total training time: 0.12 seconds.
-- Epoch 6
Norm: 9.58, NNZs: 1005, Bias: -1.501700, T: 504000, Avg. loss: 0.039489
Total training time: 0.13 seconds.
-- Epoch 8
Norm: 8.83, NNZs: 1005, Bias: -1.429865, T: 360000, Avg. loss: 0.053878
Total training time: 0.08 seconds.
-- Epoch 6
Norm: 10.61, NNZs: 1005, Bias: -1.460956, T: 432000, Avg. loss: 0.045989
Total training time: 0.12 seconds.
-- Epoch 7
Norm: 8.62, NNZs: 1005, Bias: -1.389063, T: 432000, Avg. loss: 0.053548
Total training time: 0.10 seconds.
-- Epoch 7
Norm: 9.44, NNZs: 1005, Bias: -1.470113, T

[Parallel(n_jobs=-3)]: Done  45 out of  45 | elapsed:    1.5s finished


SGDClassifier(loss='modified_huber', n_jobs=-3, random_state=2, verbose=5)

In [82]:
sgd.score(prepared_test_df,y_test)

0.6068888888888889

In [83]:
grid_sgd=GridSearchCV(sgd,{'loss': ['modified_huber'],'alpha':np.arange(0.00005,0.0002,0.00001)},
                       cv=6,n_jobs=-3,verbose=5)

In [84]:
grid_sgd.fit(prepared_train_df,y_train)

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 6 folds for each of 15 candidates, totalling 90 fits


[Parallel(n_jobs=-3)]: Done   6 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-3)]: Done  60 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-3)]: Done  90 out of  90 | elapsed:  1.9min finished
[Parallel(n_jobs=-3)]: Using backend ThreadingBackend with 6 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1

Norm: 25.12, NNZs: 1005, Bias: -2.549884, T: 72000, Avg. loss: 0.224806Norm: 20.77, NNZs: 972, Bias: -3.279087, T: 72000, Avg. loss: 0.080296
Total training time: 0.02 seconds.

Total training time: 0.03 seconds.
-- Epoch 2
-- Epoch 2
Norm: 21.44, NNZs: 1003, Bias: -2.609902, T: 72000, Avg. loss: 0.222490
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 21.22, NNZs: 1004, Bias: -2.305645, T: 72000, Avg. loss: 0.191255
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 24.88, NNZs: 1000, Bias: -3.668388, T: 72000, Avg. loss: 0.119908
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 24.75, NNZs: 1004, Bias: -3.170412, T: 72000, Avg. loss: 0.152668
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 17.87, NNZs: 1005, Bias: -1.826335, T: 144000, Avg. loss: 0.080670Norm: 16.81, NNZs: 991, Bias: -2.735261, T: 144000, Avg. loss: 0.030363
Total training time: 0.05 seconds.

Total training time: 0.04 seconds.
-- Epoch 3
-

[Parallel(n_jobs=-3)]: Done   6 tasks      | elapsed:    0.2s


...

Norm: 11.60, NNZs: 1005, Bias: -1.503918, T: 360000, Avg. loss: 0.062191
Total training time: 0.11 seconds.
-- Epoch 6
Norm: 7.15, NNZs: 1005, Bias: -1.056308, T: 432000, Avg. loss: 0.081450
Total training time: 0.12 seconds.
-- Epoch 7
Norm: 11.29, NNZs: 1005, Bias: -1.101950, T: 648000, Avg. loss: 0.060840
Total training time: 0.18 seconds.
-- Epoch 10
Norm: 23.82, NNZs: 1001, Bias: -3.294978, T: 72000, Avg. loss: 0.118090
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 19.28, NNZs: 1005, Bias: -3.177777, T: 216000, Avg. loss: 0.041891
Total training time: 0.05 seconds.
-- Epoch 4
Norm: 9.62, NNZs: 1005, Bias: -1.119239, T: 576000, Avg. loss: 0.068510
Total training time: 0.14 seconds.
-- Epoch 9
Norm: 6.89, NNZs: 1005, Bias: -1.033369, T: 504000, Avg. loss: 0.080665
Total training time: 0.15 seconds.
-- Epoch 8
Norm: 11.19, NNZs: 1005, Bias: -1.445681, T: 432000, Avg. loss: 0.060968
Total training time: 0.13 seconds.
-- Epoch 7
Norm: 11.17, NNZs: 1005, Bias: -1.102999, T: 720000

Norm: 22.75, NNZs: 999, Bias: -2.982225, T: 72000, Avg. loss: 0.099418
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 23.77, NNZs: 990, Bias: -3.123755, T: 72000, Avg. loss: 0.129824
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 9.46, NNZs: 1005, Bias: -1.165093, T: 504000, Avg. loss: 0.065136
Total training time: 0.14 seconds.
-- Epoch 8
Norm: 19.42, NNZs: 1004, Bias: -1.938278, T: 216000, Avg. loss: 0.040073
Total training time: 0.05 seconds.
-- Epoch 4
Norm: 23.47, NNZs: 1005, Bias: -2.653453, T: 72000, Avg. loss: 0.164265
Total training time: 0.02 seconds.
Norm: 10.47, NNZs: 1005, Bias: -1.219034, T: 360000, Avg. loss: 0.068599
-- Epoch 2
Total training time: 0.10 seconds.
-- Epoch 6
Norm: 17.96, NNZs: 1003, Bias: -2.360659, T: 144000, Avg. loss: 0.033893
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 9.26, NNZs: 1005, Bias: -1.141788, T: 576000, Avg. loss: 0.064687
Total training time: 0.17 seconds.Norm: 18.24, NNZs: 1005, Bias: -1.791266, T: 288000, Avg. loss: 0.0

Norm: 19.80, NNZs: 999, Bias: -2.363051, T: 144000, Avg. loss: 0.054540
Total training time: 0.05 seconds.
Norm: 10.15, NNZs: 1005, Bias: -1.125257, T: 720000, Avg. loss: 0.065888-- Epoch 3

Total training time: 0.22 seconds.
-- Epoch 11
Norm: 14.10, NNZs: 1005, Bias: -1.905326, T: 432000, Avg. loss: 0.039107
Total training time: 0.12 seconds.
-- Epoch 7
Norm: 16.73, NNZs: 1005, Bias: -2.329173, T: 144000, Avg. loss: 0.067915
Total training time: 0.04 seconds.
-- Epoch 3
Norm: 12.60, NNZs: 1004, Bias: -1.856370, T: 720000, Avg. loss: 0.036874
Total training time: 0.21 seconds.
Convergence after 10 epochs took 0.21 seconds
Norm: 17.31, NNZs: 1002, Bias: -2.087087, T: 216000, Avg. loss: 0.048513
Total training time: 0.05 seconds.
-- Epoch 4Norm: 10.03, NNZs: 1005, Bias: -1.120067, T: 792000, Avg. loss: 0.065782

Total training time: 0.22 seconds.
Convergence after 11 epochs took 0.22 seconds
Norm: 13.83, NNZs: 1005, Bias: -1.829089, T: 504000, Avg. loss: 0.038624
Total training time: 0.1

[Parallel(n_jobs=-3)]: Done  45 out of  45 | elapsed:    1.5s finished


GridSearchCV(cv=6,
             estimator=SGDClassifier(loss='modified_huber', n_jobs=-3,
                                     random_state=2, verbose=5),
             n_jobs=-3,
             param_grid={'alpha': array([5.0e-05, 6.0e-05, 7.0e-05, 8.0e-05, 9.0e-05, 1.0e-04, 1.1e-04,
       1.2e-04, 1.3e-04, 1.4e-04, 1.5e-04, 1.6e-04, 1.7e-04, 1.8e-04,
       1.9e-04]),
                         'loss': ['modified_huber']},
             verbose=5)

In [85]:
grid_sgd.score(prepared_test_df,y_test)

0.6108333333333333

In [86]:
grid_sgd.best_params_

{'alpha': 5e-05, 'loss': 'modified_huber'}

#### Perceptron

In [87]:
pc=Perceptron(verbose=1,n_jobs=-2,max_iter=100000,n_iter_no_change=5,random_state=2)


In [88]:
pc.fit(prepared_train_df,y_train)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


-- Epoch 1-- Epoch 1
-- Epoch 1

-- Epoch 1
-- Epoch 1-- Epoch 1
-- Epoch 1

Norm: 15.57, NNZs: 933, Bias: -0.850000, T: 72000, Avg. loss: 0.005257
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 18.01, NNZs: 999, Bias: -0.730000, T: 72000, Avg. loss: 0.012898
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 20.37, NNZs: 998, Bias: -0.880000, T: 72000, Avg. loss: 0.013962
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 18.07, NNZs: 982, Bias: -0.910000, T: 72000, Avg. loss: 0.014713Norm: 18.94, NNZs: 989, Bias: -0.900000, T: 72000, Avg. loss: 0.009671
Total training time: 0.03 seconds.
-- Epoch 2

Total training time: 0.03 seconds.
-- Epoch 2
Norm: 17.34, NNZs: 985, Bias: -0.910000, T: 72000, Avg. loss: 0.006924
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 19.65, NNZs: 995, Bias: -1.000000, T: 72000, Avg. loss: 0.010036
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 18.91, NNZs: 959, Bias: -1.070000, T: 144000, Avg. loss: 0.004938
Total training time: 0.05 second

Norm: 16.24, NNZs: 1002, Bias: -0.850000, T: 72000, Avg. loss: 0.018125
Total training time: 0.04 seconds.
-- Epoch 2
Norm: 23.80, NNZs: 987, Bias: -1.310000, T: 216000, Avg. loss: 0.003850
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 22.30, NNZs: 1000, Bias: -0.880000, T: 144000, Avg. loss: 0.013610Norm: 20.50, NNZs: 967, Bias: -1.010000, T: 216000, Avg. loss: 0.003488

Total training time: 0.05 seconds.
-- Epoch 3
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 20.58, NNZs: 1003, Bias: -0.910000, T: 144000, Avg. loss: 0.015590
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 19.39, NNZs: 1004, Bias: -0.800000, T: 144000, Avg. loss: 0.017762
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 20.88, NNZs: 1002, Bias: -1.060000, T: 216000, Avg. loss: 0.016770
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 17.70, NNZs: 1002, Bias: -0.980000, T: 144000, Avg. loss: 0.019307
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 24.96, NNZs: 1002, Bias: -0.980000, T: 216000

-- Epoch 1
-- Epoch 1
Norm: 27.43, NNZs: 995, Bias: -1.290000, T: 360000, Avg. loss: 0.005039
Total training time: 0.10 seconds.
-- Epoch 6
Norm: 18.46, NNZs: 997, Bias: -0.810000, T: 72000, Avg. loss: 0.010891
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 23.32, NNZs: 1005, Bias: -1.160000, T: 432000, Avg. loss: 0.015251
Total training time: 0.15 seconds.
Convergence after 6 epochs took 0.15 seconds
-- Epoch 1Norm: 26.91, NNZs: 991, Bias: -1.350000, T: 288000, Avg. loss: 0.006907

Total training time: 0.11 seconds.
-- Epoch 5
Norm: 37.06, NNZs: 995, Bias: -1.470000, T: 432000, Avg. loss: 0.005969
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 18.18, NNZs: 1000, Bias: -0.850000, T: 72000, Avg. loss: 0.010002
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 22.68, NNZs: 1003, Bias: -1.040000, T: 144000, Avg. loss: 0.010443
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 29.20, NNZs: 997, Bias: -1.350000, T: 432000, Avg. loss: 0.005087
Total training time: 0.14 seconds.

[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:    0.9s finished


Perceptron(max_iter=100000, n_jobs=-2, random_state=2, verbose=1)

In [89]:
pc.score(prepared_test_df,y_test)

0.5095555555555555

In [90]:
grid_pc=GridSearchCV(pc,{'penalty':['l2',],'alpha': np.arange(0.00001,0.0004,0.0001),},cv=6,n_jobs=-2,verbose=5)


In [91]:
grid_pc.fit(prepared_train_df,y_train)

Fitting 6 folds for each of 4 candidates, totalling 24 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-2)]: Done  16 out of  24 | elapsed:   15.2s remaining:    7.6s
[Parallel(n_jobs=-2)]: Done  21 out of  24 | elapsed:   21.0s remaining:    2.9s
[Parallel(n_jobs=-2)]: Done  24 out of  24 | elapsed:   21.1s finished
[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


-- Epoch 1-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1

-- Epoch 1
Norm: 16.08, NNZs: 1002, Bias: -0.770000, T: 72000, Avg. loss: 0.012896
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 14.71, NNZs: 992, Bias: -0.700000, T: 72000, Avg. loss: 0.011988Norm: 12.30, NNZs: 936, Bias: -0.830000, T: 72000, Avg. loss: 0.004961
Total training time: 0.02 seconds.

-- Epoch 2
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 13.86, NNZs: 986, Bias: -0.860000, T: 72000, Avg. loss: 0.006432
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 14.81, NNZs: 990, Bias: -0.840000, T: 72000, Avg. loss: 0.009073
Total training time: 0.02 seconds.
Norm: 14.34, NNZs: 986, Bias: -0.830000, T: 72000, Avg. loss: 0.013724-- Epoch 2
Total training time: 0.03 seconds.

-- Epoch 2
Norm: 15.56, NNZs: 994, Bias: -0.920000, T: 72000, Avg. loss: 0.009488
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 16.93, NNZs: 1003, Bias: -0.900000, T: 144000, Avg. loss: 0.013139
Total training time: 0.06 seco

Norm: 13.74, NNZs: 988, Bias: -1.050000, T: 288000, Avg. loss: 0.003721
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 14.98, NNZs: 1005, Bias: -0.920000, T: 288000, Avg. loss: 0.015612
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 14.51, NNZs: 1003, Bias: -0.870000, T: 144000, Avg. loss: 0.017594
Total training time: 0.06 seconds.
Norm: 14.89, NNZs: 1005, Bias: -0.910000, T: 216000, Avg. loss: 0.014311-- Epoch 3
Total training time: 0.07 seconds.
-- Epoch 4

Norm: 14.51, NNZs: 1005, Bias: -0.820000, T: 288000, Avg. loss: 0.016552
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 12.49, NNZs: 983, Bias: -0.880000, T: 288000, Avg. loss: 0.003428
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 13.90, NNZs: 990, Bias: -1.090000, T: 360000, Avg. loss: 0.003713
Total training time: 0.11 seconds.
-- Epoch 6
Norm: 16.14, NNZs: 1003, Bias: -0.820000, T: 288000, Avg. loss: 0.013026
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 14.65, NNZs: 1005, Bias: -0.920000, T: 360000

-- Epoch 5
Norm: 15.45, NNZs: 1001, Bias: -0.870000, T: 144000, Avg. loss: 0.009920
Norm: 14.17, NNZs: 1002, Bias: -0.980000, T: 432000, Avg. loss: 0.005008Total training time: 0.04 seconds.
Total training time: 0.14 seconds.

-- Epoch 3
Convergence after 6 epochs took 0.14 seconds
Norm: 11.63, NNZs: 954, Bias: -1.070000, T: 432000, Avg. loss: 0.003532
Total training time: 0.18 seconds.
-- Epoch 1Convergence after 6 epochs took 0.18 seconds

Norm: 15.57, NNZs: 1002, Bias: -0.970000, T: 144000, Avg. loss: 0.009142Norm: 14.70, NNZs: 1001, Bias: -1.080000, T: 360000, Avg. loss: 0.006517
Total training time: 0.11 seconds.
-- Epoch 6
-- Epoch 1
Total training time: 0.04 seconds.

-- Epoch 3
Norm: 17.63, NNZs: 1001, Bias: -1.000000, T: 360000, Avg. loss: 0.006294
Total training time: 0.14 seconds.
-- Epoch 6
Norm: 15.28, NNZs: 1002, Bias: -0.910000, T: 216000, Avg. loss: 0.009995
Norm: 15.54, NNZs: 1005, Bias: -0.870000, T: 432000, Avg. loss: 0.013639Total training time: 0.07 seconds.
-- Epo

[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:    0.9s finished


GridSearchCV(cv=6,
             estimator=Perceptron(max_iter=100000, n_jobs=-2, random_state=2,
                                  verbose=1),
             n_jobs=-2,
             param_grid={'alpha': array([1.0e-05, 1.1e-04, 2.1e-04, 3.1e-04]),
                         'penalty': ['l2']},
             verbose=5)

In [92]:
grid_pc.score(prepared_test_df,y_test)

0.47

In [93]:
grid_pc.best_params_

{'alpha': 1e-05, 'penalty': 'l2'}

In [94]:
pc=Perceptron(verbose=1,n_jobs=-2,max_iter=100000,n_iter_no_change=5,random_state=2)

#### PassiveAgressive

In [95]:
pa=PassiveAggressiveClassifier(verbose=1,n_jobs=-2,max_iter=100000,n_iter_no_change=5,random_state=2)


In [96]:
pa.fit(prepared_train_df,y_train)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1-- Epoch 1

-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 24.25, NNZs: 1005, Bias: -1.361670, T: 71997, Avg. loss: 0.058526
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 26.53, NNZs: 1005, Bias: -1.535336, T: 71997, Avg. loss: 0.064624
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 26.66, NNZs: 1005, Bias: -1.579723, T: 71997, Avg. loss: 0.048859
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 21.97, NNZs: 1004, Bias: -1.454304, T: 71997, Avg. loss: 0.028422
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 24.77, NNZs: 1005, Bias: -1.653489, T: 71997, Avg. loss: 0.037161
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 23.26, NNZs: 1005, Bias: -1.518866, T: 71997, Avg. loss: 0.064608
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 26.87, NNZs: 1005, Bias: -1.708190, T: 71997, Avg. loss: 0.051104
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 31.01, NNZs: 1005, Bias: -1.642631, T: 143994, Avg. loss: 0.054503
Total training time: 0.0

-- Epoch 7
Norm: 36.45, NNZs: 1005, Bias: -2.151119, T: 431982, Avg. loss: 0.064794
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 43.93, NNZs: 1005, Bias: -1.858843, T: 431982, Avg. loss: 0.045086
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 43.39, NNZs: 1005, Bias: -2.772159, T: 431982, Avg. loss: 0.013924
Total training time: 0.13 seconds.
-- Epoch 7
Norm: 51.13, NNZs: 1005, Bias: -2.887468, T: 575976, Avg. loss: 0.039400
Total training time: 0.20 seconds.
-- Epoch 9
Norm: 53.56, NNZs: 1005, Bias: -2.288433, T: 503979, Avg. loss: 0.038301Norm: 32.82, NNZs: 1005, Bias: -1.806358, T: 287988, Avg. loss: 0.010523
Total training time: 0.18 seconds.
-- Epoch 8

Total training time: 0.09 seconds.
-- Epoch 5
Norm: 37.55, NNZs: 1005, Bias: -2.231922, T: 503979, Avg. loss: 0.065086
Total training time: 0.18 seconds.
Convergence after 7 epochs took 0.18 seconds
Norm: 42.15, NNZs: 1005, Bias: -2.347044, T: 503979, Avg. loss: 0.052408
Total training time: 0.17 seconds.
Convergence af

Norm: 43.57, NNZs: 1005, Bias: -1.953400, T: 359985, Avg. loss: 0.051885
Total training time: 0.12 seconds.
-- Epoch 6
Norm: 43.20, NNZs: 1005, Bias: -2.704543, T: 359985, Avg. loss: 0.040328
Total training time: 0.12 seconds.
-- Epoch 6
Norm: 43.50, NNZs: 1005, Bias: -2.412432, T: 359985, Avg. loss: 0.044162
Total training time: 0.13 seconds.
-- Epoch 6
Norm: 38.57, NNZs: 1005, Bias: -2.399325, T: 503979, Avg. loss: 0.054886
Total training time: 0.16 seconds.
Convergence after 7 epochs took 0.16 seconds
Norm: 42.81, NNZs: 1005, Bias: -2.376945, T: 359985, Avg. loss: 0.038562
Total training time: 0.11 seconds.
-- Epoch 6
-- Epoch 1
Norm: 46.20, NNZs: 1005, Bias: -2.767080, T: 431982, Avg. loss: 0.030540
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 47.36, NNZs: 1005, Bias: -3.499544, T: 503979, Avg. loss: 0.032809
Total training time: 0.17 seconds.
-- Epoch 8
Norm: 46.28, NNZs: 1005, Bias: -2.042316, T: 431982, Avg. loss: 0.051426
Total training time: 0.15 seconds.
-- Epoch 7
Nor

Norm: 52.66, NNZs: 1005, Bias: -2.419162, T: 575976, Avg. loss: 0.043472
Total training time: 0.18 seconds.
-- Epoch 9
-- Epoch 7
Norm: 28.80, NNZs: 1005, Bias: -1.808658, T: 143994, Avg. loss: 0.053829
Total training time: 0.05 seconds.
Norm: 29.14, NNZs: 1005, Bias: -1.966811, T: 143994, Avg. loss: 0.037065
Total training time: 0.05 seconds.
-- Epoch 3
-- Epoch 3
Norm: 46.93, NNZs: 1005, Bias: -2.994655, T: 359985, Avg. loss: 0.039889
Total training time: 0.11 seconds.
-- Epoch 6
Norm: 34.34, NNZs: 1005, Bias: -1.987197, T: 287988, Avg. loss: 0.060889
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 45.17, NNZs: 1005, Bias: -2.542405, T: 431982, Avg. loss: 0.053061
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 54.33, NNZs: 1005, Bias: -2.558497, T: 647973, Avg. loss: 0.043337
Total training time: 0.21 seconds.
Convergence after 9 epochs took 0.21 seconds
-- Epoch 1
Norm: 50.75, NNZs: 1005, Bias: -2.812634, T: 503979, Avg. loss: 0.039497
Total training time: 0.16 seconds.
-- 

[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:    1.3s finished


PassiveAggressiveClassifier(max_iter=100000, n_jobs=-2, random_state=2,
                            verbose=1)

In [97]:
pa.score(prepared_test_df,y_test)

0.5802777777777778

In [107]:
grid_pa=GridSearchCV(pa,{'C': np.arange(0.1,2,0.1),},cv=6,n_jobs=-1,verbose=5)


In [108]:
grid_pa.fit(prepared_train_df,y_train)

Fitting 6 folds for each of 19 candidates, totalling 114 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 114 out of 114 | elapsed: 15.6min finished
[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


-- Epoch 1-- Epoch 1
-- Epoch 1
-- Epoch 1-- Epoch 1


-- Epoch 1
-- Epoch 1
Norm: 10.40, NNZs: 1005, Bias: -0.918394, T: 71997, Avg. loss: 0.048903
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 7.91, NNZs: 1005, Bias: -0.978130, T: 71997, Avg. loss: 0.051672
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 11.89, NNZs: 1005, Bias: -0.935204, T: 71997, Avg. loss: 0.026643Norm: 13.12, NNZs: 1005, Bias: -0.966243, T: 71997, Avg. loss: 0.044638
Total training time: 0.03 seconds.
-- Epoch 2

Total training time: 0.03 seconds.
-- Epoch 2
Norm: 13.50, NNZs: 1005, Bias: -1.013226, T: 71997, Avg. loss: 0.034557
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 11.60, NNZs: 1005, Bias: -1.023945, T: 71997, Avg. loss: 0.045131
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 8.07, NNZs: 1005, Bias: -0.934335, T: 71997, Avg. loss: 0.051553
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 9.89, NNZs: 1005, Bias: -1.138530, T: 143994, Avg. loss: 0.048312Norm: 13.37, NNZs: 1005, Bia

-- Epoch 14
Norm: 28.82, NNZs: 1005, Bias: -1.732145, T: 935961, Avg. loss: 0.041014
Total training time: 0.37 seconds.
-- Epoch 14
Norm: 38.26, NNZs: 1005, Bias: -2.242271, T: 1007958, Avg. loss: 0.031453
Total training time: 0.37 seconds.
-- Epoch 15
Norm: 32.81, NNZs: 1005, Bias: -2.045054, T: 1007958, Avg. loss: 0.015904
Total training time: 0.37 seconds.
-- Epoch 15
Norm: 22.66, NNZs: 1005, Bias: -1.680684, T: 935961, Avg. loss: 0.046763
Total training time: 0.37 seconds.
-- Epoch 14
Norm: 34.20, NNZs: 1005, Bias: -2.217476, T: 935961, Avg. loss: 0.021674
Total training time: 0.38 seconds.
-- Epoch 14
Norm: 39.81, NNZs: 1005, Bias: -2.544147, T: 1079955, Avg. loss: 0.032102
Total training time: 0.38 seconds.
-- Epoch 16
Norm: 27.28, NNZs: 1005, Bias: -1.874083, T: 1007958, Avg. loss: 0.045894
Total training time: 0.40 seconds.
-- Epoch 15
Norm: 29.44, NNZs: 1005, Bias: -1.775130, T: 1007958, Avg. loss: 0.040955
Total training time: 0.40 seconds.
-- Epoch 15
Norm: 39.35, NNZs: 1005

Norm: 37.08, NNZs: 1005, Bias: -2.173192, T: 2015916, Avg. loss: 0.045195
Total training time: 0.75 seconds.
Norm: 53.99, NNZs: 1005, Bias: -3.104383, T: 2159910, Avg. loss: 0.030083
Total training time: 0.74 seconds.
Norm: 45.79, NNZs: 1005, Bias: -2.604466, T: 2159910, Avg. loss: 0.014297-- Epoch 31

Total training time: 0.74 seconds.
-- Epoch 29
Norm: 51.89, NNZs: 1005, Bias: -2.766466, T: 2159910, Avg. loss: 0.029464-- Epoch 31

Total training time: 0.75 seconds.
-- Epoch 31
Norm: 36.94, NNZs: 1005, Bias: -1.965960, T: 1943919, Avg. loss: 0.040372
Total training time: 0.75 seconds.
-- Epoch 28
Norm: 45.90, NNZs: 1005, Bias: -2.757923, T: 2015916, Avg. loss: 0.020235
Total training time: 0.75 seconds.
-- Epoch 29
Norm: 31.16, NNZs: 1005, Bias: -1.904598, T: 2015916, Avg. loss: 0.046239
Total training time: 0.76 seconds.
-- Epoch 29
Norm: 46.48, NNZs: 1005, Bias: -2.626905, T: 2231907, Avg. loss: 0.014194Norm: 37.62, NNZs: 1005, Bias: -2.197866, T: 2087913, Avg. loss: 0.045175
Total 

...

-- Epoch 33
Norm: 31.19, NNZs: 1005, Bias: -1.906045, T: 2735886, Avg. loss: 0.048006
Total training time: 0.92 seconds.
-- Epoch 39
Norm: 22.14, NNZs: 1005, Bias: -1.681426, T: 503979, Avg. loss: 0.029178
Total training time: 0.16 seconds.
-- Epoch 8
Norm: 37.41, NNZs: 1005, Bias: -2.181838, T: 2519895, Avg. loss: 0.040930
Total training time: 0.88 seconds.
-- Epoch 36
Norm: 28.23, NNZs: 1005, Bias: -1.724803, T: 431982, Avg. loss: 0.033512
Total training time: 0.13 seconds.
-- Epoch 7
Norm: 61.31, NNZs: 1004, Bias: -3.662900, T: 3167868, Avg. loss: 0.029949
Total training time: 1.06 seconds.
-- Epoch 45
Norm: 21.77, NNZs: 1004, Bias: -1.641543, T: 431982, Avg. loss: 0.041003
Total training time: 0.14 seconds.
-- Epoch 7
Norm: 47.21, NNZs: 1005, Bias: -2.680815, T: 2375901, Avg. loss: 0.025984
Total training time: 0.82 seconds.
-- Epoch 34
Norm: 31.56, NNZs: 1005, Bias: -1.900496, T: 2807883, Avg. loss: 0.047988
Total training time: 0.94 seconds.
-- Epoch 40
Norm: 23.52, NNZs: 1005, B

-- Epoch 21
Norm: 34.68, NNZs: 1005, Bias: -1.984925, T: 3599850, Avg. loss: 0.048068
Total training time: 1.29 seconds.
-- Epoch 51
Norm: 40.45, NNZs: 1005, Bias: -2.228601, T: 3311862, Avg. loss: 0.040593
Total training time: 1.25 seconds.
-- Epoch 47
Norm: 35.27, NNZs: 1004, Bias: -2.332108, T: 1439940, Avg. loss: 0.038654
Total training time: 0.50 seconds.
-- Epoch 21
Norm: 53.35, NNZs: 1005, Bias: -2.901129, T: 3239865, Avg. loss: 0.025502
Total training time: 1.18 seconds.
-- Epoch 46
Norm: 38.15, NNZs: 1005, Bias: -2.415006, T: 1583934, Avg. loss: 0.026905
Total training time: 0.55 seconds.
-- Epoch 23
Norm: 47.53, NNZs: 1005, Bias: -2.566477, T: 1511937, Avg. loss: 0.029616
Total training time: 0.53 seconds.
-- Epoch 22
Norm: 40.52, NNZs: 1005, Bias: -2.253238, T: 3383859, Avg. loss: 0.040682
Total training time: 1.28 seconds.
-- Epoch 48
Norm: 35.68, NNZs: 1004, Bias: -2.398007, T: 1511937, Avg. loss: 0.038680
Total training time: 0.53 seconds.
-- Epoch 22
Norm: 34.93, NNZs: 1

-- Epoch 45
Norm: 53.54, NNZs: 1005, Bias: -3.056570, T: 3311862, Avg. loss: 0.025580
Total training time: 1.10 seconds.
-- Epoch 47
Norm: 64.86, NNZs: 1005, Bias: -3.185976, T: 3239865, Avg. loss: 0.027768
Total training time: 1.07 seconds.
-- Epoch 46
Norm: 45.42, NNZs: 1004, Bias: -2.750111, T: 3239865, Avg. loss: 0.037930
Total training time: 1.08 seconds.
-- Epoch 46
Norm: 53.99, NNZs: 1005, Bias: -3.060313, T: 3383859, Avg. loss: 0.025482
Total training time: 1.12 seconds.
-- Epoch 48
Norm: 65.50, NNZs: 1005, Bias: -3.178619, T: 3311862, Avg. loss: 0.027692
Total training time: 1.09 seconds.
-- Epoch 47
Norm: 45.65, NNZs: 1004, Bias: -2.786543, T: 3311862, Avg. loss: 0.037944Norm: 54.58, NNZs: 1005, Bias: -3.074608, T: 3455856, Avg. loss: 0.025596
Total training time: 1.10 seconds.

Total training time: 1.13 seconds.
-- Epoch 47
-- Epoch 49
Norm: 65.92, NNZs: 1005, Bias: -3.230246, T: 3383859, Avg. loss: 0.027708
Total training time: 1.11 seconds.
-- Epoch 48
Norm: 55.00, NNZs: 1

[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:    8.5s finished


GridSearchCV(cv=6,
             estimator=PassiveAggressiveClassifier(C=0.1, max_iter=1000000,
                                                   n_iter_no_change=50,
                                                   n_jobs=-2, random_state=2,
                                                   verbose=1),
             n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9])},
             verbose=5)

In [109]:
grid_pa.score(prepared_test_df,y_test)

0.6026111111111111

In [110]:
grid_pa.best_score_

0.5962916666666667

In [111]:
grid_pa.best_params_

{'C': 0.2}

In [112]:
pa=PassiveAggressiveClassifier(verbose=1,C=0.2,n_jobs=-2,max_iter=1000000,n_iter_no_change=50,random_state=2)

In [113]:
pa.fit(prepared_train_df,y_train)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


-- Epoch 1
-- Epoch 1
-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1
-- Epoch 1

Norm: 11.60, NNZs: 1005, Bias: -1.023945, T: 71997, Avg. loss: 0.045131
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 11.89, NNZs: 1005, Bias: -0.935204, T: 71997, Avg. loss: 0.026643
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 7.91, NNZs: 1005, Bias: -0.978130, T: 71997, Avg. loss: 0.051672
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 13.50, NNZs: 1005, Bias: -1.013226, T: 71997, Avg. loss: 0.034557Norm: 13.12, NNZs: 1005, Bias: -0.966243, T: 71997, Avg. loss: 0.044638
Total training time: 0.03 seconds.

Total training time: 0.03 seconds.
-- Epoch 2
-- Epoch 2
Norm: 8.07, NNZs: 1005, Bias: -0.934335, T: 71997, Avg. loss: 0.051553
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 10.40, NNZs: 1005, Bias: -0.918394, T: 71997, Avg. loss: 0.048903
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 14.71, NNZs: 1005, Bias: -1.297757, T: 143994, Avg. loss: 0.039438
Total training time: 0.05 

Norm: 37.02, NNZs: 1005, Bias: -2.193968, T: 935961, Avg. loss: 0.031495
Total training time: 0.37 seconds.
-- Epoch 14
Norm: 31.71, NNZs: 1005, Bias: -2.146565, T: 791967, Avg. loss: 0.021988
Total training time: 0.38 seconds.
-- Epoch 12
Norm: 22.66, NNZs: 1005, Bias: -1.680684, T: 935961, Avg. loss: 0.046763
Total training time: 0.38 seconds.
-- Epoch 14
Norm: 38.44, NNZs: 1005, Bias: -2.492876, T: 1007958, Avg. loss: 0.032140
Total training time: 0.39 seconds.
-- Epoch 15
Norm: 26.39, NNZs: 1005, Bias: -1.829662, T: 935961, Avg. loss: 0.046112
Total training time: 0.39 seconds.
-- Epoch 14
Norm: 27.96, NNZs: 1005, Bias: -1.711700, T: 863964, Avg. loss: 0.041082
Total training time: 0.38 seconds.
-- Epoch 13
Norm: 32.81, NNZs: 1005, Bias: -2.045054, T: 1007958, Avg. loss: 0.015904
Total training time: 0.39 seconds.
-- Epoch 15
Norm: 38.26, NNZs: 1005, Bias: -2.242271, T: 1007958, Avg. loss: 0.031453
Total training time: 0.40 seconds.
-- Epoch 15
Norm: 23.35, NNZs: 1005, Bias: -1.719

-- Epoch 23
Norm: 35.39, NNZs: 1005, Bias: -2.135151, T: 1799925, Avg. loss: 0.045288
Total training time: 0.75 seconds.
-- Epoch 26
Norm: 35.56, NNZs: 1005, Bias: -1.939126, T: 1727928, Avg. loss: 0.040521
Total training time: 0.74 seconds.
-- Epoch 25
Norm: 29.41, NNZs: 1005, Bias: -1.884227, T: 1727928, Avg. loss: 0.046334
Total training time: 0.75 seconds.
-- Epoch 25
Norm: 43.81, NNZs: 1005, Bias: -2.515105, T: 1943919, Avg. loss: 0.014424
Total training time: 0.76 seconds.
-- Epoch 28
Norm: 51.60, NNZs: 1005, Bias: -3.032479, T: 1943919, Avg. loss: 0.030302
Total training time: 0.76 seconds.
-- Epoch 28
Norm: 42.54, NNZs: 1005, Bias: -2.612077, T: 1655931, Avg. loss: 0.020529
Total training time: 0.76 seconds.
-- Epoch 24
Norm: 35.97, NNZs: 1005, Bias: -2.130520, T: 1871922, Avg. loss: 0.045279
Total training time: 0.77 seconds.
-- Epoch 27
Norm: 36.07, NNZs: 1005, Bias: -1.946312, T: 1799925, Avg. loss: 0.040400
Total training time: 0.77 seconds.
-- Epoch 26
Norm: 49.83, NNZs: 1

Norm: 41.12, NNZs: 1005, Bias: -2.279493, T: 2591892, Avg. loss: 0.044887
Total training time: 1.12 seconds.
-- Epoch 37
Norm: 61.03, NNZs: 1005, Bias: -3.371130, T: 2951877, Avg. loss: 0.029349
Total training time: 1.13 seconds.
-- Epoch 42
Norm: 57.65, NNZs: 1005, Bias: -2.974507, T: 2879880, Avg. loss: 0.029012
Total training time: 1.12 seconds.
-- Epoch 41
Norm: 51.71, NNZs: 1005, Bias: -2.855027, T: 2879880, Avg. loss: 0.013707
Total training time: 1.13 seconds.
-- Epoch 41
Norm: 40.51, NNZs: 1005, Bias: -2.007204, T: 2591892, Avg. loss: 0.040187
Total training time: 1.12 seconds.
-- Epoch 37
Norm: 49.89, NNZs: 1005, Bias: -2.925486, T: 2519895, Avg. loss: 0.019797
Total training time: 1.14 seconds.
-- Epoch 36
Norm: 34.06, NNZs: 1005, Bias: -1.973988, T: 2591892, Avg. loss: 0.046262
Total training time: 1.14 seconds.
-- Epoch 37
Norm: 41.29, NNZs: 1005, Bias: -2.333033, T: 2663889, Avg. loss: 0.045028
Total training time: 1.15 seconds.
-- Epoch 38
Norm: 61.58, NNZs: 1005, Bias: -

...

-- Epoch 29
Norm: 56.15, NNZs: 1004, Bias: -3.444502, T: 2591892, Avg. loss: 0.030349
Total training time: 1.15 seconds.
-- Epoch 37
Norm: 42.93, NNZs: 1005, Bias: -2.512524, T: 1871922, Avg. loss: 0.026341
Total training time: 0.75 seconds.
-- Epoch 27
Norm: 31.96, NNZs: 1005, Bias: -1.872201, T: 575976, Avg. loss: 0.032422
Total training time: 0.20 seconds.
-- Epoch 9
Norm: 28.22, NNZs: 1005, Bias: -1.797660, T: 2159910, Avg. loss: 0.048119
Total training time: 0.89 seconds.
-- Epoch 31
Norm: 21.77, NNZs: 1004, Bias: -1.641543, T: 431982, Avg. loss: 0.041003
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 30.63, NNZs: 1005, Bias: -2.088231, T: 1007958, Avg. loss: 0.027772
Total training time: 0.36 seconds.
-- Epoch 15
Norm: 35.19, NNZs: 1005, Bias: -2.122335, T: 2087913, Avg. loss: 0.041016
Total training time: 0.89 seconds.
-- Epoch 30
Norm: 33.61, NNZs: 1005, Bias: -1.951856, T: 647973, Avg. loss: 0.032138
Total training time: 0.22 seconds.
-- Epoch 10
Norm: 43.68, NNZs: 1005, 

-- Epoch 40
Norm: 46.63, NNZs: 1005, Bias: -2.500384, T: 1439940, Avg. loss: 0.029778
Total training time: 0.57 seconds.
-- Epoch 21
Norm: 62.98, NNZs: 1004, Bias: -3.742082, T: 3383859, Avg. loss: 0.029779
Total training time: 1.53 seconds.
-- Epoch 48
Norm: 31.85, NNZs: 1005, Bias: -1.900989, T: 2879880, Avg. loss: 0.048058Norm: 33.91, NNZs: 1004, Bias: -2.279940, T: 1295946, Avg. loss: 0.038859
Total training time: 1.27 seconds.

Total training time: 0.53 seconds.
-- Epoch 41
-- Epoch 19
Norm: 41.29, NNZs: 1005, Bias: -2.539315, T: 1871922, Avg. loss: 0.026684
Total training time: 0.74 seconds.
-- Epoch 27
Norm: 49.84, NNZs: 1005, Bias: -2.789099, T: 2735886, Avg. loss: 0.025649
Total training time: 1.15 seconds.
-- Epoch 39
Norm: 38.83, NNZs: 1005, Bias: -2.215986, T: 2879880, Avg. loss: 0.040694
Total training time: 1.27 seconds.
-- Epoch 41
Norm: 63.65, NNZs: 1004, Bias: -3.753657, T: 3455856, Avg. loss: 0.029688
Total training time: 1.55 seconds.
-- Epoch 49
Norm: 47.53, NNZs: 1

-- Epoch 37
Norm: 41.47, NNZs: 1004, Bias: -2.609741, T: 2375901, Avg. loss: 0.038106
Norm: 56.76, NNZs: 1005, Bias: -3.023135, T: 3815841, Avg. loss: 0.025215
Total training time: 1.50 seconds.
Total training time: 0.89 seconds.
Convergence after 53 epochs took 1.50 seconds
-- Epoch 34
Norm: 51.50, NNZs: 1005, Bias: -2.952276, T: 3023874, Avg. loss: 0.025827
Total training time: 1.10 seconds.
-- Epoch 43
Norm: 60.04, NNZs: 1005, Bias: -3.018216, T: 2663889, Avg. loss: 0.028255
Total training time: 0.96 seconds.
-- Epoch 38
Norm: 41.85, NNZs: 1004, Bias: -2.624867, T: 2447898, Avg. loss: 0.038117
Total training time: 0.91 seconds.
-- Epoch 35
Norm: 52.03, NNZs: 1005, Bias: -2.975446, T: 3095871, Avg. loss: 0.025757
Total training time: 1.12 seconds.
-- Epoch 44
Norm: 60.75, NNZs: 1005, Bias: -3.026796, T: 2735886, Avg. loss: 0.028094
Total training time: 0.97 seconds.
-- Epoch 39
Norm: 42.18, NNZs: 1004, Bias: -2.653459, T: 2519895, Avg. loss: 0.038073
Total training time: 0.92 seconds

[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:   10.7s finished


PassiveAggressiveClassifier(C=0.2, max_iter=1000000, n_iter_no_change=50,
                            n_jobs=-2, random_state=2, verbose=1)

In [114]:
pa.score(prepared_test_df,y_test)

0.6026111111111111

In [115]:
prepared_train_df.shape

(72000, 1005)

#### MultiLayerPerceptron

In [243]:
mlp=MLPClassifier(verbose=1,hidden_layer_sizes=(80,42,),n_iter_no_change=10,random_state=2,max_iter=600)


In [244]:
mlp.fit(prepared_train_df,y_train)

Iteration 1, loss = 3.37827026
Iteration 2, loss = 2.00154234
Iteration 3, loss = 1.65378861
Iteration 4, loss = 1.53415407
Iteration 5, loss = 1.46328188
Iteration 6, loss = 1.41057242
Iteration 7, loss = 1.36943338
Iteration 8, loss = 1.33401633
Iteration 9, loss = 1.30373435
Iteration 10, loss = 1.27521945
Iteration 11, loss = 1.24742824
Iteration 12, loss = 1.22243539
Iteration 13, loss = 1.19953949
Iteration 14, loss = 1.17644795
Iteration 15, loss = 1.15461737
Iteration 16, loss = 1.13336701
Iteration 17, loss = 1.11299726
Iteration 18, loss = 1.09308401
Iteration 19, loss = 1.07464615
Iteration 20, loss = 1.05487455
Iteration 21, loss = 1.03636511
Iteration 22, loss = 1.01937552
Iteration 23, loss = 1.00095609
Iteration 24, loss = 0.98336179
Iteration 25, loss = 0.96793593
Iteration 26, loss = 0.95085974
Iteration 27, loss = 0.93563963
Iteration 28, loss = 0.91949589
Iteration 29, loss = 0.90262374
Iteration 30, loss = 0.88858489
Iteration 31, loss = 0.87262316
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(80, 42), max_iter=600, random_state=2,
              verbose=1)

In [245]:
mlp.score(prepared_test_df,y_test)

0.6189366659606016

In [246]:
grid_mlp=GridSearchCV(mlp,{'beta_1':[0.8,0.9,0.99],'beta_2':[0.799,0.899,0.999]},cv=6,n_jobs=-3,verbose=5)


In [247]:
grid_mlp.fit(prepared_train_df,y_train)

Fitting 6 folds for each of 9 candidates, totalling 54 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done   6 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-3)]: Done  54 out of  54 | elapsed: 81.0min remaining:    0.0s
[Parallel(n_jobs=-3)]: Done  54 out of  54 | elapsed: 81.0min finished


Iteration 1, loss = 3.58503378
Iteration 2, loss = 2.51338807
Iteration 3, loss = 1.86145753
Iteration 4, loss = 1.65793395
Iteration 5, loss = 1.55967540
Iteration 6, loss = 1.49325557
Iteration 7, loss = 1.44308004
Iteration 8, loss = 1.40173171
Iteration 9, loss = 1.36738701
Iteration 10, loss = 1.33642242
Iteration 11, loss = 1.30703808
Iteration 12, loss = 1.28116824
Iteration 13, loss = 1.25742719
Iteration 14, loss = 1.23431133
Iteration 15, loss = 1.21193987
Iteration 16, loss = 1.19094168
Iteration 17, loss = 1.17079198
Iteration 18, loss = 1.15062778
Iteration 19, loss = 1.13254110
Iteration 20, loss = 1.11318842
Iteration 21, loss = 1.09504787
Iteration 22, loss = 1.07817314
Iteration 23, loss = 1.05993559
Iteration 24, loss = 1.04233267
Iteration 25, loss = 1.02698458
Iteration 26, loss = 1.01067794
Iteration 27, loss = 0.99481515
Iteration 28, loss = 0.97908992
Iteration 29, loss = 0.96252741
Iteration 30, loss = 0.94859516
Iteration 31, loss = 0.93320490
Iteration 32, los

GridSearchCV(cv=6,
             estimator=MLPClassifier(hidden_layer_sizes=(80, 42), max_iter=600,
                                     random_state=2, verbose=1),
             n_jobs=-3,
             param_grid={'beta_1': [0.8, 0.9, 0.99],
                         'beta_2': [0.799, 0.899, 0.999]},
             verbose=5)

In [248]:
grid_mlp.score(prepared_test_df,y_test)

0.6165007413683542

In [249]:
grid_mlp.best_params_

{'beta_1': 0.9, 'beta_2': 0.799}

In [250]:
mlp=MLPClassifier(verbose=1,n_jobs=-2,random_state=2)


TypeError: __init__() got an unexpected keyword argument 'n_jobs'

#### Réseau de neurones.

In [252]:
prepared_train_array=prepared_train_df.toarray()
prepared_test_array=prepared_test_df.toarray()

In [253]:
train_tensor =tf.convert_to_tensor(prepared_train_array.reshape(prepared_train_array.shape[0],prepared_train_array.shape[1],1))
test_tensor=tf.convert_to_tensor(prepared_test_array.reshape(prepared_test_array.shape[0],prepared_test_array.shape[1],1))

In [254]:
len(col_proc.transformers[0][1].steps[0][1].vocabulary_)

991

In [314]:
tf.keras.backend.clear_session()
model_nn=Sequential()
model_nn.add(Input(shape=(prepared_train_array.shape[1]),))
#model_nn.add(Embedding(input_dim=len(col_proc.transformers[0][1].steps[0][1].vocabulary_),output_dim=1))
#model_nn.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),)

#model_nn.add(Embedding(2,1000))
#model_nn.add(Flatten())
model_nn.add(Dense(904))
model_nn.add(Dense(1500))
#model_nn.add(SimpleRNNCell(10))
model_nn.add(Dense(2000))
model_nn.add(Dense(1500))
model_nn.add(Dense(500 ))
#model_nn.add(Dense(200 ))
#model_nn.add(Dense(104 ))
#model_nn.add(Embedding(104,200))
#model_nn.add(Embedding(1000,250))
#model_nn.add(Conv2D(160, 6, 
#            activation='relu',))

#model_nn.add(Embedding(4,3))
#model_nn.add(Conv1D(600, 5, 
#            activation='relu',))
#model_nn.add(Conv1D(600, 3, 
#            activation='relu',))
#model_nn.add(LSTM(52))
#model_nn.add(Dense(1))
#model_nn.add(Flatten())

model_nn.add(Dense(1000))
model_nn.add(Dense(500 ))

model_nn.add(Dense(52))

model_nn.add(Dense(y.max()))

In [315]:
model_nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 904)               904000    
_________________________________________________________________
dense_1 (Dense)              (None, 1500)              1357500   
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              3002000   
_________________________________________________________________
dense_3 (Dense)              (None, 1500)              3001500   
_________________________________________________________________
dense_4 (Dense)              (None, 500)               750500    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_6 (Dense)              (None, 500)               5

In [316]:
model_nn.build()
model_nn.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])

In [317]:
lr_reduce=tf.keras.callbacks.ReduceLROnPlateau(
    monitor='accuracy', factor=0.1, patience=3, verbose=1,
    mode='auto', min_delta=0.001, cooldown=5, min_lr=0, 
)

In [318]:
y_binarytr = to_categorical(y_train)
y_binaryt =to_categorical(y_test)

In [319]:
model_nn.fit(prepared_train_array,y_train,batch_size=7000,workers=6,epochs=250,callbacks=[lr_reduce])

Train on 37767 samples
Epoch 1/250
37767/37767 [==============================] - 3s 76us/sample - loss: nan - accuracy: 0.0209
Epoch 2/250
37767/37767 [==============================] - 2s 50us/sample - loss: nan - accuracy: 0.0211
Epoch 3/250
37767/37767 [==============================] - 2s 46us/sample - loss: nan - accuracy: 0.0211
Epoch 4/250
35000/37767 [==========================>...] - ETA: 0s - loss: nan - accuracy: 0.0215
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
37767/37767 [==============================] - 2s 46us/sample - loss: nan - accuracy: 0.0211
Epoch 5/250
37767/37767 [==============================] - 2s 47us/sample - loss: nan - accuracy: 0.0211
Epoch 6/250
37767/37767 [==============================] - 2s 47us/sample - loss: nan - accuracy: 0.0211
Epoch 7/250
37767/37767 [==============================] - 2s 48us/sample - loss: nan - accuracy: 0.0211
Epoch 8/250
37767/37767 [==============================] - 2s 49us/sample -

KeyboardInterrupt: 

In [287]:
y_pred=model_nn.predict_classes(prepared_test_array,verbose=1)
accuracy_score(y_test,y_pred)

9442/9442 [==============================] - 1s 104us/sample


0.5343147638212243

In [354]:
y_binaryt.shape

(5720, 52)

In [604]:
prepared_train_df

array([[-1.01498525e+00,  1.48900861e-01, -1.56913350e-01, ...,
        -5.97850021e-03,  1.65048779e-02, -1.26535994e-02],
       [ 1.03488117e-01,  1.33916289e-01,  3.27174205e-02, ...,
        -3.35252712e-16, -3.36749988e-16,  6.92508162e-17],
       [ 2.34043484e+00,  1.35911197e-01, -5.37132008e-02, ...,
        -1.62114031e-02, -1.50691723e-02, -1.86570082e-03],
       ...,
       [ 1.22196148e+00,  1.86217738e-01,  5.72965822e-02, ...,
         2.92722826e-02,  5.33726012e-02,  2.44535263e-01],
       [-1.01498525e+00,  7.46638872e-02, -8.99150110e-02, ...,
         7.52082598e-03,  8.50874559e-03,  3.40171220e-03],
       [-1.01498525e+00,  1.58216062e-01, -5.37441120e-02, ...,
        -2.37424047e-02, -1.23467818e-02, -3.00693448e-02]])

In [463]:
mlp.score(prepared_test_df,y_test)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 881 is different from 817)

In [390]:
grid_mlp=GridSearchCV(mlp,{'beta_1':[0.8,0.9,0.99],'beta_2':[0.799,0.899,0.999]},cv=6,n_jobs=-3,verbose=5)


In [ ]:
grid_mlp.fit(prepared_train_df,y_train)

Fitting 6 folds for each of 9 candidates, totalling 54 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.


In [ ]:
grid_mlp.score(prepared_test_df,y_test)

In [ ]:
grid_mlp.best_params_

In [76]:
mlp=MLPClassifier(verbose=1,n_jobs=-2,random_state=2)


## Modèle retenu

In [ ]:
X_train['Title']

In [ ]:
word_proc.fit(X_train[words_columns])

In [ ]:
col_proc

In [ ]:
(tsvd.fit_transform,tsvd.fit_transform)(x,x)

In [ ]:
word_proc.fit(X_train[['Body_text']])

In [ ]:
query